In [ ]:
from shared.logging_config import setup_logging
setup_logging()

In [ ]:
from streaming import binance_websocket, config

client = binance_websocket.BinanceWebSocketClient(
    symbols=["BTCUSDT", "ETHUSDT"],
    streams=["trade", "kline_1m"],
    callback=lambda data: print(data),  # Example callback function
    config=config.StreamingConfig()
)


In [ ]:
# Start the WebSocket client
await client.connect()

In [ ]:
client._extract_symbol("stream=btcusdt@trade")  # Example usage of _extract_symbol method

In [ ]:
client.